In [1]:
import os
import random
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import difflib

In [2]:
dataset_path = "../../data/raw/" 

### ***Database table relationships (from kaggle)***
<img src="https://i.imgur.com/HRhd2Y0.png" alt="Database table relationships" style="height: 500px; width:900px;"/>

In [3]:
files = os.listdir(dataset_path)
print(f'The dataset contains {len(files)} files:')
for file in files:
    print(f'    * {file}')

The dataset contains 9 files:
    * olist_customers_dataset.csv
    * olist_geolocation_dataset.csv
    * olist_orders_dataset.csv
    * olist_order_items_dataset.csv
    * olist_order_payments_dataset.csv
    * olist_order_reviews_dataset.csv
    * olist_products_dataset.csv
    * olist_sellers_dataset.csv
    * product_category_name_translation.csv


In [4]:
def tables_info(df):
    # csv_file_name = 'olist_customers_dataset.csv'
    # csv_file_path = os.path.join(dataset_path, csv_file_name)
    # df = pd.read_csv(csv_file_path)
    # print(df.head(5))
    # print(df.columns.shape[0])    
    info_df = pd.DataFrame(data=[], index=df.columns)            
    #info_df.loc[:, 'table_name'] = csv_file_name.split('.')[0]
    #info_df.loc[:, 'entities'] = info_df['table_name'].str.split('_dataset').str[0].str.split('olist_').str[-1]
    info_df.loc[:, 'column_name'] = df.columns
    for column in df.columns:           
        info_df.loc[column, 'dtype'] = df[column].dtypes   
        info_df.loc[column,'rows_count'] = df[column].shape[0]
        info_df.loc[column,'non_null_rows_count'] =df[column].notnull().sum()
        info_df.loc[column,'null_row_count'] =df[column].isna().sum()
        info_df.loc[column,'unique_value_count'] = df[column].unique().shape[0]

    info_df.reset_index(drop=True, inplace=True)        
    return info_df

In [5]:
def data_subtype(df, column):
    # column = df.columns[4]
    # print(column)
    df = df[df[column].notnull()].reset_index(drop=True) #Remove possible not null values
    # dtype
    # print(df[column].dtype)
    if df[column].dtype == 'object':    
        #Date/time or str?
        try: #Date/time: Up to now only datetime  
            x = pd.to_datetime(df[column], infer_datetime_format=True)                   
            data_subtype = x.dtype

        except: #if it fails -> str
            if df[column].str.len().unique().shape[0] == 1: #All the elements have the same length if == 1
                data_subtype = f"CHAR({df[column].str.len().unique()[0]})"
            elif df[column].map(lambda x: len(x)).max() > 255: #Variable length higher than 255
                data_subtype = 'TEXT()' 
            else:
                # data_type = f"VARCHAR({df[column].map(lambda x: len(x)).max()})"   
                data_subtype = f"VARCHAR(255)"       
    elif df[column].dtype == 'float64':
        # print(df[column].dtype)
        data_subtype = df[column].dtype
    else: #int64
        # print(df[column].dtype)
        data_subtype = df[column].dtype
    # print(data_subtype)
    example = df[column][random.randint(0,df[column].shape[0])]
    return [data_subtype, example]

In [6]:
def tables_info2(df):
    # csv_file_name = 'olist_customers_dataset.csv'
    # csv_file_path = os.path.join(dataset_path, csv_file_name)
    # df = pd.read_csv(csv_file_path)
    # print(df.head(5))
    # print(df.columns.shape[0])    
    info_df = pd.DataFrame(data=[], index=df.columns)            
    #info_df.loc[:, 'table_name'] = csv_file_name.split('.')[0]
    #info_df.loc[:, 'entities'] = info_df['table_name'].str.split('_dataset').str[0].str.split('olist_').str[-1]
    info_df.loc[:, 'column_name'] = df.columns
    for column in df.columns:           
        info_df.loc[column, 'dtype'] = df[column].dtypes
        data_subtype_example = data_subtype(df, column)
        info_df.loc[column, 'subtype'] = data_subtype_example[0]
        info_df.loc[column, 'example'] = data_subtype_example[1]
        info_df.loc[column,'unique_value_count'] = df[column].unique().shape[0]
        info_df.loc[column,'is_not_null'] =~df[column].isnull().values.any()
        info_df.loc[column, 'is_unique'] = df[column].is_unique        
        info_df.loc[column, 'is_primary_key'] = (column.find('_id') != -1) & info_df.loc[column,'is_not_null'] & info_df.loc[column, 'is_unique']
        info_df.loc[column, 'is_foreign_key'] = (column.find('_id') != -1) & ~info_df.loc[column, 'is_primary_key']
        info_df.loc[column,'is_id'] = (column.find('_id') != -1)       

    info_df.reset_index(drop=True, inplace=True)        
    return info_df

In [7]:
csv_file_name = 'olist_geolocation_dataset.csv'
csv_file_path = os.path.join(dataset_path, csv_file_name)
df = pd.read_csv(csv_file_path)

In [8]:
info_df = tables_info(df)
info_df

,column_name,dtype,rows_count,non_null_rows_count,null_row_count,unique_value_count
0,geolocation_zip_code_prefix,int64,1000163.0,1000163.0,0.0,19015.0
1,geolocation_lat,float64,1000163.0,1000163.0,0.0,717360.0
2,geolocation_lng,float64,1000163.0,1000163.0,0.0,717613.0
3,geolocation_city,object,1000163.0,1000163.0,0.0,8011.0
4,geolocation_state,object,1000163.0,1000163.0,0.0,27.0


In [9]:
info_df = tables_info2(df)
info_df

,column_name,dtype,subtype,example,unique_value_count,is_not_null,is_unique,is_primary_key,is_foreign_key,is_id
0,geolocation_zip_code_prefix,int64,int64,3122.0,19015.0,True,False,False,False,False
1,geolocation_lat,float64,float64,-23.551289,717360.0,True,False,False,False,False
2,geolocation_lng,float64,float64,-49.055862,717613.0,True,False,False,False,False
3,geolocation_city,object,VARCHAR(255),santo andré,8011.0,True,False,False,False,False
4,geolocation_state,object,CHAR(2),SP,27.0,True,False,False,False,False


## Comments:
This dataset has ***NO NULL VALUES!***


In [10]:
df.head()

,geolocation_zip_code_prefix,geolocation_lat,geolocation_lng,geolocation_city,geolocation_state
0,1037,-23.545621,-46.639292,sao paulo,SP
1,1046,-23.546081,-46.644820,sao paulo,SP
2,1046,-23.546129,-46.642951,sao paulo,SP
3,1041,-23.544392,-46.639499,sao paulo,SP
4,1035,-23.541578,-46.641607,sao paulo,SP


#### Each `geolocation_zip_code_prefix` must be assigned to only one `geolocation_city` and only one `geolocation_state`

In [11]:
# Get the list of unique zip_code_prefix_list
geolocation_zip_code_prefix_list = df['geolocation_zip_code_prefix'].sort_values().unique().tolist()#[0:10]
len(geolocation_zip_code_prefix_list)

19015

#### For each `geolocation_zip_code_prefix` we create a new dataframe containing the following columns:
* `zip_code_prefix` (which is unique)
* `state`
* `most_frequent_city` (which is the most repeated citiy for the corresponding `zip_code_prefix`. Regardless it is well written or  not)
* `city_list` (list of unique cities matching a given `zip_code_prefix`)
* `city_number` (number of cities for each city_list)

In [12]:
# zip_code = 24220
geolocation_zip_code_prefix_list = df['geolocation_zip_code_prefix'].sort_values().unique().tolist()#[0:1000]
cities_per_zip_df = pd.DataFrame(columns=['zip_code_prefix', 'state', 'most_frequent_city', 'city_list'])
for zip_code in geolocation_zip_code_prefix_list:
    # print(zip_code)
    df_per_zip = df[df['geolocation_zip_code_prefix'] == zip_code]
    # New row to add (per each zip_code)
    new_row = pd.DataFrame(columns=cities_per_zip_df.columns)
    new_row.loc[0, 'zip_code_prefix'] = zip_code
    new_row.loc[0, 'state'] = df_per_zip['geolocation_state'].value_counts(ascending=False).index[0]
    new_row.loc[0, 'most_frequent_city'] = df_per_zip['geolocation_city'].value_counts(ascending=False).index[0] #The most repeated value
    geolocation_city_list = df_per_zip['geolocation_city'].unique().tolist()
    geolocation_city_list.sort(key = str)    
    new_row.loc[0, 'city_list'] = geolocation_city_list
    new_row.loc[0, 'city_number'] = int(len(new_row.loc[0, 'city_list']))
    cities_per_zip_df = pd.concat([cities_per_zip_df, new_row], axis=0, ignore_index=True)    
cities_per_zip_df.head()

,zip_code_prefix,state,most_frequent_city,city_list,city_number
0,1001,SP,sao paulo,"[sao paulo, são paulo]",2.0
1,1002,SP,sao paulo,"[sao paulo, são paulo]",2
2,1003,SP,sao paulo,"[sao paulo, são paulo]",2
3,1004,SP,sao paulo,"[sao paulo, são paulo]",2
4,1005,SP,sao paulo,"[sao paulo, são paulo]",2


In [13]:
# Take the most_frequent_city column and remove portuguese special characters
cities_per_zip_df['most_frequent_city'] = cities_per_zip_df['most_frequent_city'].str.normalize('NFKD').str.encode('ascii', errors='ignore').str.decode('utf-8')
# cities_per_zip_df.to_csv('cities_per_zip.csv', sep=',', index=False, encoding='utf-8-sig')
cities_per_zip_df.head()

,zip_code_prefix,state,most_frequent_city,city_list,city_number
0,1001,SP,sao paulo,"[sao paulo, são paulo]",2.0
1,1002,SP,sao paulo,"[sao paulo, são paulo]",2
2,1003,SP,sao paulo,"[sao paulo, são paulo]",2
3,1004,SP,sao paulo,"[sao paulo, são paulo]",2
4,1005,SP,sao paulo,"[sao paulo, são paulo]",2


In [14]:
# Create a dataframe from brazil_cities.csv
brazil_cities_df = pd.read_csv('../../data/external/brazil_cities.csv').sort_values('code').reset_index(drop=True)
brazil_cities_df.head()

,code,name,state
0,1100015,alta floresta d oeste,RO
1,1100023,ariquemes,RO
2,1100031,cabixi,RO
3,1100049,cacoal,RO
4,1100056,cerejeiras,RO


In [15]:
# Function to get the best match for a list of possible cities
def get_close_match(city_str, posibble_cities_list):
    best_match = difflib.get_close_matches(city_str, posibble_cities_list, cutoff=0.85)
    if not best_match:
        #print("list is  empty")
        city_name_corr = city_str
    else:
        #print("list is not empty")
        city_name_corr = best_match[0]
    return city_name_corr

In [16]:
# Iterate over rows to get a column with corrected city values 'most_frequent_city_corr'
for index, row in cities_per_zip_df.iterrows():
    #print(row['most_frequent_city'])   
    state = row['state']
    city_str = row['most_frequent_city']
    state_cities_df = brazil_cities_df[brazil_cities_df['state'] == state]
    posibble_cities_list = state_cities_df['name'].to_list()
    city_name_corr = get_close_match(city_str, posibble_cities_list)
    cities_per_zip_df.loc[index, 'most_frequent_city_corr'] = city_name_corr

In [17]:
cities_per_zip_df.head()

,zip_code_prefix,state,most_frequent_city,city_list,city_number,most_frequent_city_corr
0,1001,SP,sao paulo,"[sao paulo, são paulo]",2.0,sao paulo
1,1002,SP,sao paulo,"[sao paulo, são paulo]",2,sao paulo
2,1003,SP,sao paulo,"[sao paulo, são paulo]",2,sao paulo
3,1004,SP,sao paulo,"[sao paulo, são paulo]",2,sao paulo
4,1005,SP,sao paulo,"[sao paulo, são paulo]",2,sao paulo


In [18]:
# Create a dictionary containing zip_code_prefix as key and most_frequent_city_corr as value
zip_code_to_city_dict = dict(zip(cities_per_zip_df['zip_code_prefix'].to_list(), cities_per_zip_df['most_frequent_city_corr'].to_list()))
# Uncommnet to see the resulting dictionary
#zip_code_to_city_dict

In [19]:
#cities_per_zip_df[['zip_code_prefix','most_frequent_city_corr', 'state']].to_csv('../../data/interim/zip_code_to_city.csv', sep=',', index=False, encoding='utf-8-sig')

In [20]:
# Apply the dictionary to the original dataframe
df['geolocation_city_new'] = df['geolocation_zip_code_prefix'].map(zip_code_to_city_dict)
# Rename geolocation_city columns
df.rename( columns={'geolocation_city' : 'geolocation_city_old',  'geolocation_city_new' : 'geolocation_city'} ,inplace=True) 
df.head()

,geolocation_zip_code_prefix,geolocation_lat,geolocation_lng,geolocation_city_old,geolocation_state,geolocation_city
0,1037,-23.545621,-46.639292,sao paulo,SP,sao paulo
1,1046,-23.546081,-46.644820,sao paulo,SP,sao paulo
2,1046,-23.546129,-46.642951,sao paulo,SP,sao paulo
3,1041,-23.544392,-46.639499,sao paulo,SP,sao paulo
4,1035,-23.541578,-46.641607,sao paulo,SP,sao paulo


In [21]:
# Rows who had originally different city names
df[df['geolocation_city_old'] != df['geolocation_city']]

,geolocation_zip_code_prefix,geolocation_lat,geolocation_lng,geolocation_city_old,geolocation_state,geolocation_city
5,1012,-23.547762,-46.635361,são paulo,SP,sao paulo
14,1037,-23.545187,-46.637855,são paulo,SP,sao paulo
17,1024,-23.541390,-46.629899,são paulo,SP,sao paulo
21,1020,-23.552235,-46.628441,são paulo,SP,sao paulo
22,1011,-23.546690,-46.635447,são paulo,SP,sao paulo
...,...,...,...,...,...,...
1000094,99940,-28.060955,-51.858637,ibiaçá,RS,ibiaca
1000096,99900,-27.884844,-52.230025,getúlio vargas,RS,getulio vargas
1000098,99900,-27.900022,-52.237668,getúlio vargas,RS,getulio vargas
1000143,99930,-27.913659,-52.248615,estação,RS,estacao


In [22]:
df_to_csv = df[['geolocation_zip_code_prefix', 'geolocation_lat', 'geolocation_lng', 'geolocation_city', 'geolocation_state']]
df_to_csv.head()

,geolocation_zip_code_prefix,geolocation_lat,geolocation_lng,geolocation_city,geolocation_state
0,1037,-23.545621,-46.639292,sao paulo,SP
1,1046,-23.546081,-46.644820,sao paulo,SP
2,1046,-23.546129,-46.642951,sao paulo,SP
3,1041,-23.544392,-46.639499,sao paulo,SP
4,1035,-23.541578,-46.641607,sao paulo,SP


### Next step: Check if all the zip_codes and cities are here
### Check olist_customers_dataset.csv and olist_sellers_dataset.csv as well